# CDC State and County Policy Measures

* [COVID-19 State and County Policy Orders](https://healthdata.gov/dataset/covid-19-state-and-county-policy-orders)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re
from bs4 import BeautifulSoup
import requests

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
y, m, d = datetime.now().year, datetime.now().month, datetime.now().day
datasource_site = "https://healthdata.gov/dataset/covid-19-state-and-county-policy-orders"
response = requests.get(datasource_site)
assert response.status_code == 200
soup = BeautifulSoup(response.content)
link = soup.find("a", {"class": "data-link"})
assert link
url = link["href"]

In [ ]:
df = pd.read_csv(url)

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]
df["ISO3166_1"] = "US"
df["ISO3166_2"] = df["state_id"]
df["DATE"] = f"{y}-{m}-{d}"
df = df[df["ISO3166_2"].isin(states)]

In [ ]:
df.to_csv(output_folder + "CDC_POLICY_MEASURES.csv", index=False, columns=df.columns)